# Benchmark on a given set of problems

In this example, we show how to make a loop over the problems in the benchmark. We also show how to run various reliability algorithms on a given problem so that we can score the methods using number of digits or performance.

In [1]:
import openturns as ot
import numpy as np
import otbenchmark as otb
import openturns.viewer as otv
import pylab as pl

## Browse the reliability problems

We present the BBRC test cases using the otbenchmark module. 

In [2]:
benchmarkProblemList = otb.ReliabilityBenchmarkProblemList()
numberOfProblems = len(benchmarkProblemList)
numberOfProblems

26

In [3]:
for i in range(numberOfProblems):
    problem = benchmarkProblemList[i]
    name = problem.getName()
    pf = problem.getProbability()
    event = problem.getEvent()
    antecedent = event.getAntecedent()
    distribution = antecedent.getDistribution()
    dimension = distribution.getDimension()
    print("#", i, ":", name, " : pf = ", pf, ", dimension=", dimension)

# 0 : RP8  : pf =  0.000784 , dimension= 6
# 1 : RP14  : pf =  0.00752 , dimension= 5
# 2 : RP22  : pf =  0.00416 , dimension= 2
# 3 : RP24  : pf =  0.00286 , dimension= 2
# 4 : RP25  : pf =  6.14e-06 , dimension= 2
# 5 : RP28  : pf =  1.46e-07 , dimension= 2
# 6 : RP31  : pf =  0.00018 , dimension= 2
# 7 : RP33  : pf =  0.00257 , dimension= 3
# 8 : RP35  : pf =  0.00354 , dimension= 2
# 9 : RP38  : pf =  0.0081 , dimension= 7
# 10 : RP53  : pf =  0.0313 , dimension= 2
# 11 : RP55  : pf =  0.36 , dimension= 2
# 12 : RP54  : pf =  0.000998 , dimension= 20
# 13 : RP57  : pf =  0.0284 , dimension= 2
# 14 : RP75  : pf =  0.0107 , dimension= 2
# 15 : RP89  : pf =  0.00543 , dimension= 2
# 16 : RP107  : pf =  2.92e-07 , dimension= 10
# 17 : RP110  : pf =  3.19e-05 , dimension= 2
# 18 : RP111  : pf =  7.65e-07 , dimension= 2
# 19 : RP63  : pf =  0.000379 , dimension= 100
# 20 : RP91  : pf =  0.000697 , dimension= 5
# 21 : RP60  : pf =  0.0456 , dimension= 5
# 22 : RP77  : pf =  2.87e-07 , dim

In [4]:
maximumEvaluationNumber = 1000
maximumAbsoluteError = 1.0e-3
maximumRelativeError = 1.0e-3
maximumResidualError = 1.0e-3
maximumConstraintError = 1.0e-3
nearestPointAlgorithm = ot.AbdoRackwitz()
nearestPointAlgorithm.setMaximumCallsNumber(maximumEvaluationNumber)
nearestPointAlgorithm.setMaximumAbsoluteError(maximumAbsoluteError)
nearestPointAlgorithm.setMaximumRelativeError(maximumRelativeError)
nearestPointAlgorithm.setMaximumResidualError(maximumResidualError)
nearestPointAlgorithm.setMaximumConstraintError(maximumConstraintError)

# The FORM method

In [5]:
problem = otb.ReliabilityProblem8()

In [6]:
metaAlgorithm = otb.ReliabilityBenchmarkMetaAlgorithm(problem)

In [7]:
benchmarkResult = metaAlgorithm.runFORM(nearestPointAlgorithm)
benchmarkResult.summary()

'computedProbability = 0.000659887791408224  exactProbability = 0.000784  absoluteError = 0.00012411220859177602 numberOfCorrectDigits = 0.8005015586776216 numberOfFunctionEvaluations = 7numberOfDigitsPerEvaluation = 0.11435736552537452'

# The SORM method

In [8]:
benchmarkResult = metaAlgorithm.runSORM(nearestPointAlgorithm)
benchmarkResult.summary()

'computedProbability = 0.0007838036444007566  exactProbability = 0.000784  absoluteError = 1.9635559924335248e-07 numberOfCorrectDigits = 3.60127277263278 numberOfFunctionEvaluations = 7numberOfDigitsPerEvaluation = 0.5144675389475399'

## The LHS method

In [9]:
benchmarkResult = metaAlgorithm.runLHS(maximumOuterSampling=10000)
benchmarkResult.summary()

'computedProbability = 0.0012000000000000116  exactProbability = 0.000784  absoluteError = 0.00041600000000001163 numberOfCorrectDigits = 0.27522273205768355 numberOfFunctionEvaluations = 10000numberOfDigitsPerEvaluation = 2.7522273205768354e-05'

# The MonteCarloSampling method

In [10]:
benchmarkResult = metaAlgorithm.runMonteCarlo(maximumOuterSampling=10000)
benchmarkResult.summary()

'computedProbability = 0.0003000000000000004  exactProbability = 0.000784  absoluteError = 0.00048399999999999957 numberOfCorrectDigits = 0.2094707010400263 numberOfFunctionEvaluations = 10000numberOfDigitsPerEvaluation = 2.0947070104002628e-05'

# The FORM - Importance Sampling method

In [11]:
benchmarkResult = metaAlgorithm.runFORMImportanceSampling(nearestPointAlgorithm)
benchmarkResult.summary()

'computedProbability = 0.0008185203661905634  exactProbability = 0.000784  absoluteError = 3.4520366190563456e-05 numberOfCorrectDigits = 1.3562406686396002 numberOfFunctionEvaluations = 409numberOfDigitsPerEvaluation = 0.0033159918548645484'

# The Subset method

In [12]:
benchmarkResult = metaAlgorithm.runSubsetSampling()
benchmarkResult.summary()

'computedProbability = 0.0006124000000000045  exactProbability = 0.000784  absoluteError = 0.0001715999999999955 numberOfCorrectDigits = 0.6597987791717631 numberOfFunctionEvaluations = 20000numberOfDigitsPerEvaluation = 3.298993895858815e-05'

The following function computes the number of correct base-10 digits in the computed result compared to the exact result.

The `CompareMethods` function takes as a parameter a problem and it returns the probabilities estimated by each method. In addition, it returns the performance of these methods.

In [13]:
def PrintResults(name, benchmarkResult):
    print("------------------------------------------------------------------")
    print(name)
    numberOfDigitsPerEvaluation = (
        benchmarkResult.numberOfCorrectDigits
        / benchmarkResult.numberOfFunctionEvaluations
    )
    print("Estimated probability:", benchmarkResult.computedProbability)
    print("Number of function calls:", benchmarkResult.numberOfFunctionEvaluations)
    print("Number of correct digits=%.1f" % (benchmarkResult.numberOfCorrectDigits))
    print(
        "Performance=%.2e (correct digits/evaluation)" % (numberOfDigitsPerEvaluation)
    )
    return [name, benchmarkResult.numberOfCorrectDigits, numberOfDigitsPerEvaluation]

In [14]:
def CompareMethods(problem, nearestPointAlgorithm, maximumOuterSampling=10000):
    """
    Runs various algorithms on a given problem.
    """
    summaryList = []
    pfReference = problem.getProbability()
    print("Exact probability:", pf)
    metaAlgorithm = otb.ReliabilityBenchmarkMetaAlgorithm(problem)
    # SubsetSampling
    benchmarkResult = metaAlgorithm.runSubsetSampling()
    summaryList.append(PrintResults("SubsetSampling", benchmarkResult))
    # FORM
    benchmarkResult = metaAlgorithm.runFORM(nearestPointAlgorithm)
    summaryList.append(PrintResults("FORM", benchmarkResult))
    # SORM
    benchmarkResult = metaAlgorithm.runSORM(nearestPointAlgorithm)
    summaryList.append(PrintResults("SORM", benchmarkResult))
    # FORM - ImportanceSampling
    benchmarkResult = metaAlgorithm.runFORMImportanceSampling(
        nearestPointAlgorithm, maximumOuterSampling=maximumOuterSampling
    )
    summaryList.append(PrintResults("FORM-IS", benchmarkResult))
    # MonteCarloSampling
    benchmarkResult = metaAlgorithm.runMonteCarlo(
        maximumOuterSampling=maximumOuterSampling
    )
    summaryList.append(PrintResults("MonteCarloSampling", benchmarkResult))
    # LHS
    benchmarkResult = metaAlgorithm.runLHS()
    summaryList.append(PrintResults("LHS", benchmarkResult))
    # Gather results
    numberOfMethods = len(summaryList)
    correctDigitsList = []
    performanceList = []
    algorithmNames = []
    for i in range(numberOfMethods):
        [name, numberOfCorrectDigits, numberOfDigitsPerEvaluation] = summaryList[i]
        algorithmNames.append(name)
        correctDigitsList.append(numberOfCorrectDigits)
        performanceList.append(numberOfDigitsPerEvaluation)
    print("------------------------------------------------------------------------")
    print("Scoring by number of correct digits")
    indices = np.argsort(correctDigitsList)
    rank = list(indices)
    for i in range(numberOfMethods):
        j = rank[i]
        print("%d : %s (%.1f)" % (j, algorithmNames[j], correctDigitsList[j]))
    print("------------------------------------------------------------------------")
    print("Scoring by performance (digits/evaluation)")
    indices = np.argsort(performanceList)
    rank = list(indices)
    for i in range(len(indices)):
        j = rank[i]
        print("%d : %s (%.1e)" % (j, algorithmNames[j], performanceList[j]))
    return correctDigitsList, performanceList

In [15]:
problem = otb.ReliabilityProblem8()
_ = CompareMethods(problem, nearestPointAlgorithm)

Exact probability: 0.029198194624830955
------------------------------------------------------------------
SubsetSampling
Estimated probability: 0.0007298
Number of function calls: 20000
Number of correct digits=1.2
Performance=5.80e-05 (correct digits/evaluation)
------------------------------------------------------------------
FORM
Estimated probability: 0.000659887791408224
Number of function calls: 7
Number of correct digits=0.8
Performance=1.14e-01 (correct digits/evaluation)
------------------------------------------------------------------
SORM
Estimated probability: 0.0007838036444007566
Number of function calls: 7
Number of correct digits=3.6
Performance=5.14e-01 (correct digits/evaluation)
------------------------------------------------------------------
FORM-IS
Estimated probability: 0.0006733813101400115
Number of function calls: 415
Number of correct digits=0.9
Performance=2.05e-03 (correct digits/evaluation)
--------------------------------------------------------------

# Remarks

* We note that the FORM and SORM methods are faster, but, they do not converge to the exact proba.
* We also notice the effectiveness of the FORM-ImportanceSampling method (inexpensive method, and converges).
* The convergence of the MonteCarlo method requires a large number of simulations.
* SubsetSampling converges even if the probability is very low.
